In [160]:
import pandas as pd
import numpy as np
import random

### Downloading the data in convenient format

In [176]:
data = pd.read_csv('input_20201211_task.csv', sep=";")

data['Area, hectares'] = data['Area, hectares'].str.replace(',', '.')
data['Slope, degrees (loss coeff for SVEKLA)'] = data['Slope, degrees (loss coeff for SVEKLA)'].str.replace(',', '.')
data['Soil fertility (coeff)'] = data['Soil fertility (coeff)'].str.replace(',', '.')
data['Weed 2020 (loss coeff)'] = data['Weed 2020 (loss coeff)'].str.replace(',', '.')
data['Ownership 2020 (loss coeff)'] = data['Ownership 2020 (loss coeff)'].str.replace(',', '.')
data['Distance to asphalt, km'] = data['Distance to asphalt, km'].str.replace(',', '.')
data['Herbicide (coeff for SOYA and SVEKLA)'] = data['Herbicide (coeff for SOYA and SVEKLA)'].str.replace(',', '.')

data['Field'] = data['Field'].astype(int)
data['Area, hectares'] = data['Area, hectares'].astype(float)
data['Slope, degrees (loss coeff for SVEKLA)'] = data['Slope, degrees (loss coeff for SVEKLA)'].astype(float)
data['Soil fertility (coeff)'] = data['Soil fertility (coeff)'].astype(float)
data['Weed 2020 (loss coeff)'] = data['Weed 2020 (loss coeff)'].astype(float)
data['Ownership 2020 (loss coeff)'] = data['Ownership 2020 (loss coeff)'].astype(float)
data['Distance to asphalt, km'] = data['Distance to asphalt, km'].astype(float)
data['Herbicide (coeff for SOYA and SVEKLA)'] = data['Herbicide (coeff for SOYA and SVEKLA)'].astype(float)

data.head()

,Field,"Area, hectares","Slope, degrees (loss coeff for SVEKLA)",Soil fertility (coeff),Weed 2020 (loss coeff),Ownership 2020 (loss coeff),"Distance to asphalt, km",Cluster,Plant 2018,Plant 2019,Plant 2020,Herbicide (coeff for SOYA and SVEKLA)
0,1,125.4,0.0,1.0,0.0,0.3,0.0,Center,SVEKLA,YACHMEN,PSHENICA,0.0
1,2,154.2,0.2,1.0,0.3,0.0,0.5,Center,YACHMEN,PAR,PSHENICA,0.0
2,3,205.6,0.0,1.0,0.0,0.0,0.0,Center,SVEKLA,YACHMEN,PSHENICA,0.0
3,4,23.5,0.2,1.2,0.0,0.0,0.7,Center,YACHMEN,PAR,PSHENICA,0.0
4,5,245.4,0.0,1.0,0.0,0.3,0.0,Center,SVEKLA,YACHMEN,PSHENICA,0.0


In [177]:
plant_price = {
    "PAR": 0,
    "PSHENICA": 16000,
    "SOYA": 30000,
    "YACHMEN": 12000,
    "SVEKLA": 3500
}

plant_fertility = {
    "PAR": {"PAR": 0, "SOYA": 0, "PSHENICA": 0, "YACHMEN": 0, "SVEKLA": 0},
    "SOYA": {"PAR": 4, "SOYA": 2, "PSHENICA": 3, "YACHMEN": 4, "SVEKLA": 2},
    "PSHENICA": {"PAR": 6, "SOYA": 6, "PSHENICA": 3, "YACHMEN": 4, "SVEKLA": 5},
    "YACHMEN": {"PAR": 4, "SOYA": 5, "PSHENICA": 4, "YACHMEN": 3, "SVEKLA": 5},
    "SVEKLA": {"PAR": 40, "SOYA": 50, "PSHENICA": 60, "YACHMEN": 60, "SVEKLA": 30}
}

### Function for calculating revenue of one field 

In [178]:
def revenue(data, field, plant):
    
    field_idx = data[data['Field'] == field].index[0]
    
    # Condition №1
    plant_prev_year = data.iloc[field_idx]['Plant 2020']
    fertility_coeff = plant_fertility[plant][plant_prev_year]
    rev = data.iloc[field_idx]['Area, hectares'] * plant_price[plant] * fertility_coeff
      
    # Condition №2
    rev = rev * data.iloc[field_idx]['Soil fertility (coeff)']
    
    # Condition №3
    rev = rev * (1 - data.iloc[field_idx]['Weed 2020 (loss coeff)'])
    
    # Condition №4
    rev = rev * (1 - data.iloc[field_idx]['Ownership 2020 (loss coeff)'])
     
    # Condition №5
    if plant == "SVEKLA":
        if data.iloc[field_idx]['Distance to asphalt, km'] > 2:
            rev = rev * (1 - 0.2)
            
    # Condition №6
    if plant == "SOYA" or plant == "SVEKLA":
        rev = rev * (1 - data.iloc[field_idx]['Herbicide (coeff for SOYA and SVEKLA)'])
        
    # Condition №7
    if plant == "SVEKLA":
        rev = rev * (1 - data.iloc[field_idx]['Slope, degrees (loss coeff for SVEKLA)'])
    return rev

### Filling the original table with revenues for each field for each plant

In [179]:
rev_par = []
rev_pshenica = []
rev_soya = []
rev_yachmen = []
rev_svekla = []

for i in data['Field'].tolist():
    rev_par.append(revenue(data, i, "PAR"))
    rev_pshenica.append(revenue(data, i, "PSHENICA"))
    rev_soya.append(revenue(data, i, "SOYA"))
    rev_yachmen.append(revenue(data, i, "YACHMEN"))
    rev_svekla.append(revenue(data, i, "SVEKLA"))
    
data['PAR'] = rev_par
data['PSHENICA'] = rev_pshenica
data['SOYA'] = rev_soya
data['YACHMEN'] = rev_yachmen
data['SVEKLA'] = rev_svekla

data.head()

,Field,"Area, hectares","Slope, degrees (loss coeff for SVEKLA)",Soil fertility (coeff),Weed 2020 (loss coeff),Ownership 2020 (loss coeff),"Distance to asphalt, km",Cluster,Plant 2018,Plant 2019,Plant 2020,Herbicide (coeff for SOYA and SVEKLA),PAR,PSHENICA,SOYA,YACHMEN,SVEKLA
0,1,125.4,0.0,1.0,0.0,0.3,0.0,Center,SVEKLA,YACHMEN,PSHENICA,0.0,0.0,4213440.0,7900200.0,4213440.0,18433800.0
1,2,154.2,0.2,1.0,0.3,0.0,0.5,Center,YACHMEN,PAR,PSHENICA,0.0,0.0,5181120.0,9714600.0,5181120.0,18133920.0
2,3,205.6,0.0,1.0,0.0,0.0,0.0,Center,SVEKLA,YACHMEN,PSHENICA,0.0,0.0,9868800.0,18504000.0,9868800.0,43176000.0
3,4,23.5,0.2,1.2,0.0,0.0,0.7,Center,YACHMEN,PAR,PSHENICA,0.0,0.0,1353600.0,2538000.0,1353600.0,4737600.0
4,5,245.4,0.0,1.0,0.0,0.3,0.0,Center,SVEKLA,YACHMEN,PSHENICA,0.0,0.0,8245440.0,15460200.0,8245440.0,36073800.0


### Average production for each plant for the last 3 years

In [180]:
plants_total_2018 = data[['Area, hectares', 'Plant 2018']].groupby('Plant 2018').sum().apply(list).to_dict()
plants_total_2019 = data[['Area, hectares', 'Plant 2019']].groupby('Plant 2019').sum().apply(list).to_dict()
plants_total_2020 = data[['Area, hectares', 'Plant 2020']].groupby('Plant 2020').sum().apply(list).to_dict()

plants_avg = {}
plants_avg['PAR'] = (plants_total_2018['Area, hectares']['PAR'] + plants_total_2019['Area, hectares']['PAR'] + plants_total_2020['Area, hectares']['PAR']) / 3
plants_avg['PSHENICA'] = (plants_total_2018['Area, hectares']['PSHENICA'] + plants_total_2019['Area, hectares']['PSHENICA'] + plants_total_2020['Area, hectares']['PSHENICA']) / 3
plants_avg['SOYA'] = (plants_total_2018['Area, hectares']['SOYA'] + plants_total_2019['Area, hectares']['SOYA'] + plants_total_2020['Area, hectares']['SOYA']) / 3
plants_avg['SVEKLA'] = (plants_total_2018['Area, hectares']['SVEKLA'] + plants_total_2019['Area, hectares']['SVEKLA'] + plants_total_2020['Area, hectares']['SVEKLA']) / 3
plants_avg['YACHMEN'] = (plants_total_2018['Area, hectares']['YACHMEN'] + plants_total_2019['Area, hectares']['YACHMEN'] + plants_total_2020['Area, hectares']['YACHMEN']) / 3

plants_avg

{'PAR': 1395.0999999999997,
 'PSHENICA': 1590.8333333333333,
 'SOYA': 1346.2666666666667,
 'SVEKLA': 813.4,
 'YACHMEN': 1694.8999999999999}

### Function to create a sample of plants

In [166]:
def creating_sample_of_plants():
    
    num_to_plant = {
        0: "PAR",
        1: "PSHENICA",
        2: "SOYA",
        3: "SVEKLA",
        4: "YACHMEN"
    }
    
    rand_num_list = np.random.randint(low=5,size=59)    
    plants_sample = [num_to_plant[i] for i in rand_num_list]
    
    return plants_sample

### Function to check if number of different plants in the sample is less than 4 in each cluster

In [167]:
def cluster_vs_unique_plants(data, plants_sample):

    clusters_vs_plants = {
        'Center': [],
        'East': [],
        'Slobodka': [],
        'Far': []
    }

    for i in range(len(data)):
        cluster = data.iloc[i]['Cluster']
        if plants_sample[i] not in clusters_vs_plants[cluster]:
            clusters_vs_plants[cluster].append(plants_sample[i])
            
    trigger = True
    for k, v in clusters_vs_plants.items():
        if len(v) > 3:
            trigger = False
        
    return trigger

### Function to check if proportions of plants in the sample are correct comparing to the previous three years

In [168]:
def compare_sample_with_avg_by_plant(plants_sample, plants_avg):

    plants_total_sample = {
        'PAR': 0,
        'PSHENICA': 0,
        'SOYA': 0,
        'SVEKLA': 0,
        'YACHMEN': 0
    }

    for i in range(len(data)):
        plants_total_sample[plants_sample[i]] += data.iloc[i]['Area, hectares']

    plants_difference_sample_vs_avg = {}

    plants_difference_sample_vs_avg['PAR'] = plants_avg['PAR'] / (plants_total_sample['PAR'] + 0.001) - 1
    plants_difference_sample_vs_avg['PSHENICA'] = plants_avg['PSHENICA'] / (plants_total_sample['PSHENICA'] + 0.001) - 1
    plants_difference_sample_vs_avg['SOYA'] = plants_avg['SOYA'] / (plants_total_sample['SOYA'] + 0.001) - 1
    plants_difference_sample_vs_avg['SVEKLA'] = plants_avg['SVEKLA'] / (plants_total_sample['SVEKLA'] + 0.001) - 1
    plants_difference_sample_vs_avg['YACHMEN'] = plants_avg['YACHMEN'] / (plants_total_sample['YACHMEN'] + 0.001) - 1
    
    trigger = True
    for k, v in plants_difference_sample_vs_avg.items():
        if abs(v) > 0.1:
            trigger = False
    
    return trigger

### Function to calculate the total revenue of the sample of plants

In [169]:
def revenue_sample(plants_sample):

    revenue_summary = 0

    for i in range(len(data)):
        revenue_summary += data.iloc[i][plants_sample[i]]
    
    return revenue_summary

### Random generator of samples and selection of acceptable samples

In [170]:
count_correct_clusters = 0
count_correct_proportions = 0

samples_with_correct_clusters = []
samples_with_correct_proportions = []

for i in range(20000):
    
    plants_sample = creating_sample_of_plants()
    
    if cluster_vs_unique_plants(data, plants_sample):
        samples_with_correct_clusters.append(plants_sample)
        count_correct_clusters += 1
        
    if compare_sample_with_avg_by_plant(plants_sample, plants_avg):
        samples_with_correct_proportions.append(plants_sample)
        count_correct_proportions += 1    
        
print("Number of samples with correct limits withing clusters:", count_correct_clusters)
print("Number of samples with correct proportions of plants relative the last 3-years average:", count_correct_proportions)

Number of samples with correct limits withing clusters: 0
Number of samples with correct proportions of plants relative the last 3-years average: 21


### Choosing 3 best samples (with the highest revenue)

##### Choose from the samples with correct proportions of plants relative the last 3-years average

In [171]:
samples_with_correct_proportions_revenue = []
for s in samples_with_correct_proportions:
    samples_with_correct_proportions_revenue.append(revenue_sample(s))

best_3_idxs = np.array(samples_with_correct_proportions_revenue).argsort()[::-1][:3]

best_3_samples_with_correct_proportions = []
best_3_samples_with_correct_proportions.append(samples_with_correct_proportions[best_3_idxs[0]].copy())
best_3_samples_with_correct_proportions.append(samples_with_correct_proportions[best_3_idxs[1]].copy())
best_3_samples_with_correct_proportions.append(samples_with_correct_proportions[best_3_idxs[2]].copy())
best_3_samples_with_correct_proportions

revenue_best_3_samples_with_correct_proportions = []
for s in best_3_samples_with_correct_proportions:
    revenue_best_3_samples_with_correct_proportions.append(revenue_sample(s))    
print("Revenue for 3 best samples:", revenue_best_3_samples_with_correct_proportions)

best_3_samples_with_correct_proportions[0].append(revenue_best_3_samples_with_correct_proportions[0].copy())
best_3_samples_with_correct_proportions[1].append(revenue_best_3_samples_with_correct_proportions[1].copy())
best_3_samples_with_correct_proportions[2].append(revenue_best_3_samples_with_correct_proportions[2].copy())

Revenue for 3 best samples: [471097960.0, 456386576.0, 445517488.0]


### Saving the best samples to xlsx-file

In [174]:
fields = list(data['Field'].values)
fields.append('Revenue')

data_output = {
    'Field': fields,
    'Sample_1': best_3_samples_with_correct_proportions[0], 
    'Sample_2': best_3_samples_with_correct_proportions[1],
    'Sample_3': best_3_samples_with_correct_proportions[2]
} 

df_output = pd.DataFrame(data_output)

df_output.to_excel("output_samples_of_plants.xlsx") 

df_output

,Field,Sample_1,Sample_2,Sample_3
0,1,SVEKLA,PAR,PSHENICA
1,2,YACHMEN,PAR,YACHMEN
2,3,PSHENICA,SVEKLA,PAR
3,4,PSHENICA,YACHMEN,YACHMEN
4,5,YACHMEN,YACHMEN,PAR
5,6,SVEKLA,SOYA,PAR
6,7,PSHENICA,PSHENICA,SVEKLA
7,8,PSHENICA,SVEKLA,PSHENICA
8,9,YACHMEN,PAR,SVEKLA
9,10,PSHENICA,SOYA,PSHENICA
